In [1]:
import pandas as pd
import requests

In [2]:
# declare data sources
facebook_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRkCEf20V8iCyjql6mEHef16dhS6-ZShrxnUtauxqa6MnKeOZ7O6LyTMCDnkUrknWRIViPLp6xSha6o/pub?output=csv"
microsoft_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTKMlwwlAc8o20GW3uoE-EftfmpKy1ngsqPnAIGw_-O9EkFgY8XHcMPLwKXg0bp0kKEErLMGyp-inLC/pub?gid=0&single=true&output=csv"
tesla_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRRZpclS-nzUhfcWrvCBnvqFTrGBmaCoxRpwO5vsFNgqxWNHtEP8f4Uia9hYWN8G0S5IHeEgkg-NAxZ/pub?output=csv"

In [3]:
df_facebook = pd.read_csv(facebook_path, encoding="utf8")
# df_microsoft = pd.read_csv(microsoft_path, encoding="utf8")
# df_tesla = pd.read_csv(tesla_path, encoding="utf8")

In [4]:
df_facebook

,Date,Open,High,Low,Close,Volume
0,43889.66667,182.70,192.74,181.82,192.47,32583494
1,43892.66667,194.03,196.57,188.85,196.44,24948955
2,43893.66667,196.22,197.24,183.97,185.89,27984104
3,43894.66667,189.17,191.83,186.39,191.76,23062473
4,43895.66667,186.78,188.99,183.89,185.17,19333425
...,...,...,...,...,...,...
242,44239.66667,270.52,271.18,268.34,270.50,9097597
243,44243.66667,270.80,276.60,270.05,273.97,15417243
244,44244.66667,271.24,273.97,269.58,273.57,12763240
245,44245.66667,269.57,271.95,266.03,269.39,15249134


In [5]:
# extends field

df_facebook['year'] = pd.DatetimeIndex(df_facebook['Date']).year
df_facebook['month'] = pd.DatetimeIndex(df_facebook['Date']).month
df_facebook['weekday'] = pd.DatetimeIndex(df_facebook['Date']).dayofweek

# drop column
df_facebook = df_facebook.drop(['Date'], axis=1)

In [6]:
past_days = 7

for day in range(past_days):
  n = day + 1
  df_facebook[f"Open_lag_{n}"] = df_facebook.Open.shift(n)
  df_facebook[f"High_lag_{n}"] = df_facebook.High.shift(n)
  df_facebook[f"Low_lag_{n}"] = df_facebook.Low.shift(n)

In [7]:
df_facebook.head()

,Open,High,Low,Close,Volume,year,month,weekday,Open_lag_1,High_lag_1,Low_lag_1,Open_lag_2,High_lag_2,Low_lag_2,Open_lag_3,High_lag_3,Low_lag_3,Open_lag_4,High_lag_4,Low_lag_4,Open_lag_5,High_lag_5,Low_lag_5,Open_lag_6,High_lag_6,Low_lag_6,Open_lag_7,High_lag_7,Low_lag_7
0,182.70,192.74,181.82,192.47,32583494,1970,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,194.03,196.57,188.85,196.44,24948955,1970,1,3,182.70,192.74,181.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,196.22,197.24,183.97,185.89,27984104,1970,1,3,194.03,196.57,188.85,182.70,192.74,181.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,189.17,191.83,186.39,191.76,23062473,1970,1,3,196.22,197.24,183.97,194.03,196.57,188.85,182.70,192.74,181.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,186.78,188.99,183.89,185.17,19333425,1970,1,3,189.17,191.83,186.39,196.22,197.24,183.97,194.03,196.57,188.85,182.7,192.74,181.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# drop nan values

df_facebook = df_facebook.apply (pd.to_numeric, errors='coerce')
df_facebook = df_facebook.dropna()
df_facebook = df_facebook.reset_index(drop=True)

In [9]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
# from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
df_facebook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Open        240 non-null    float64
 1   High        240 non-null    float64
 2   Low         240 non-null    float64
 3   Close       240 non-null    float64
 4   Volume      240 non-null    int64  
 5   year        240 non-null    int64  
 6   month       240 non-null    int64  
 7   weekday     240 non-null    int64  
 8   Open_lag_1  240 non-null    float64
 9   High_lag_1  240 non-null    float64
 10  Low_lag_1   240 non-null    float64
 11  Open_lag_2  240 non-null    float64
 12  High_lag_2  240 non-null    float64
 13  Low_lag_2   240 non-null    float64
 14  Open_lag_3  240 non-null    float64
 15  High_lag_3  240 non-null    float64
 16  Low_lag_3   240 non-null    float64
 17  Open_lag_4  240 non-null    float64
 18  High_lag_4  240 non-null    float64
 19  Low_lag_4   240 non-null    f

In [11]:
X = df_facebook.iloc[:,[8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]]
y = df_facebook.iloc[:, 3]

In [12]:
X

,Open_lag_1,High_lag_1,Low_lag_1,Open_lag_2,High_lag_2,Low_lag_2,Open_lag_3,High_lag_3,Low_lag_3,Open_lag_4,High_lag_4,Low_lag_4,Open_lag_5,High_lag_5,Low_lag_5,Open_lag_6,High_lag_6,Low_lag_6,Open_lag_7,High_lag_7,Low_lag_7
0,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39,196.22,197.24,183.97,194.03,196.57,188.85,182.70,192.74,181.82
1,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39,196.22,197.24,183.97,194.03,196.57,188.85
2,174.01,175.88,167.66,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39,196.22,197.24,183.97
3,159.54,166.60,154.34,174.01,175.88,167.66,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39
4,163.53,170.79,157.60,159.54,166.60,154.34,174.01,175.88,167.66,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00,266.80,269.17,265.67,267.01,268.16,264.00,265.62,269.20,263.84
236,270.52,271.18,268.34,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00,266.80,269.17,265.67,267.01,268.16,264.00
237,270.80,276.60,270.05,270.52,271.18,268.34,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00,266.80,269.17,265.67
238,271.24,273.97,269.58,270.80,276.60,270.05,270.52,271.18,268.34,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00


In [13]:
X

,Open_lag_1,High_lag_1,Low_lag_1,Open_lag_2,High_lag_2,Low_lag_2,Open_lag_3,High_lag_3,Low_lag_3,Open_lag_4,High_lag_4,Low_lag_4,Open_lag_5,High_lag_5,Low_lag_5,Open_lag_6,High_lag_6,Low_lag_6,Open_lag_7,High_lag_7,Low_lag_7
0,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39,196.22,197.24,183.97,194.03,196.57,188.85,182.70,192.74,181.82
1,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39,196.22,197.24,183.97,194.03,196.57,188.85
2,174.01,175.88,167.66,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39,196.22,197.24,183.97
3,159.54,166.60,154.34,174.01,175.88,167.66,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89,189.17,191.83,186.39
4,163.53,170.79,157.60,159.54,166.60,154.34,174.01,175.88,167.66,174.67,178.29,169.50,169.60,175.00,165.19,178.33,183.78,176.26,186.78,188.99,183.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00,266.80,269.17,265.67,267.01,268.16,264.00,265.62,269.20,263.84
236,270.52,271.18,268.34,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00,266.80,269.17,265.67,267.01,268.16,264.00
237,270.80,276.60,270.05,270.52,271.18,268.34,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00,266.80,269.17,265.67
238,271.24,273.97,269.58,270.80,276.60,270.05,270.52,271.18,268.34,271.89,273.58,268.49,272.45,273.80,267.72,266.44,273.39,265.75,268.75,269.86,265.00


In [19]:
y

0      178.19
1      170.24
2      154.47
3      170.28
4      146.01
        ...  
235    270.50
236    273.97
237    273.57
238    269.39
239    261.56
Name: Close, Length: 240, dtype: float64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
regressorL = LinearRegression()  
regressorL.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
y_pred = regressorL.predict(X_test)

In [17]:
regressorL.coef_

array([-0.89600324,  0.60165018,  1.00431861, -0.17699922, -0.08156692,
        0.57966924,  0.54225622, -0.14630147, -0.4890261 , -0.28003001,
        0.27565845,  0.0553452 ,  0.25111677, -0.05771057, -0.19869983,
        0.0403756 , -0.09799964, -0.13175667, -0.33020987, -0.05887202,
        0.56771566])

In [18]:
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

Mean squared error: 55.29
Coefficient of determination: 0.97
